In [11]:
import os
import re
import time
import json
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext, ttk
from tkinterdnd2 import DND_FILES, TkinterDnD
from PyPDF2 import PdfReader
from openai import OpenAI
import tiktoken
import sys

# Load API Key from 'api_key.txt'
def load_api_key():
    """Reads the API key from api_key.txt in the current directory."""
    script_dir = os.getcwd()
    api_key_path = os.path.join(script_dir, "api_key.txt")

    if not os.path.exists(api_key_path):
        messagebox.showerror("Error", f"API key file not found: {api_key_path}")
        return None
    
    with open(api_key_path, "r") as file:
        return file.read().strip()

api_key = load_api_key()
if not api_key:
    exit("No API key found. Please ensure 'api_key.txt' exists in the script's directory.")

# Initialize OpenAI client
client = OpenAI(api_key=api_key)

max_length = 15000
supported_formats = {".pdf", ".txt", ".m", ".ipynb", ".py", ".doc", ".docx", ".ppt", ".pptx", ".xls", ".xlsx"}

# Redirect print statements to Tkinter text widget
class PrintRedirector:
    """Redirects print statements to a Tkinter text widget."""
    def __init__(self, text_widget):
        self.text_widget = text_widget

    def write(self, message):
        self.text_widget.insert(tk.END, message)
        self.text_widget.see(tk.END)  # Auto-scroll to the latest message

    def flush(self):
        pass  # No need to flush

def rename_file(filepath):
    """Renames a single file using OpenAI-generated names."""
    create_date = time.strftime("_%Y%m%d", time.localtime(os.path.getctime(filepath)))
    extension = os.path.splitext(filepath)[1].lower()

    print(f"📄 Reading file: {filepath}")

    file_content = extract_text_from_file(filepath)
    if not file_content:
        print(f"⚠️ Skipping {filepath} (empty or unreadable).")
        return None

    new_file_name = get_new_filename_from_openai(file_content, extension)
    new_file_name_with_date = new_file_name.replace(extension, "") + create_date + extension

    # Ensure filename is unique
    directory = os.path.dirname(filepath)
    counter = 1
    while os.path.exists(os.path.join(directory, new_file_name_with_date)):
        new_file_name_with_date = f"{new_file_name.replace(extension, '')}{create_date}_{counter}{extension}"
        counter += 1

    new_filepath = os.path.join(directory, new_file_name_with_date)

    try:
        os.rename(filepath, new_filepath)
        print(f"✅ File renamed to: {new_filepath}")
        return new_filepath
    except Exception as e:
        print(f"⚠️ Error renaming file: {e}")
        return None

def get_new_filename_from_openai(file_content, extension):
    """Generates a new filename using OpenAI."""
    response = client.chat.completions.create(
        model=selected_model.get(),  # Get the selected model from dropdown
        messages=[
            {"role": "system", "content": "Please reply with a simple and uniform filename that consists only of English characters, numbers, and underscores. If it is a SCI journal paper, the format is 'a short paper title_with_underscore_short journal name abbreviation_publish year'. If it is a book, start with 'Book'."},
            {"role": "user", "content": file_content}
        ]
    )
    initial_filename = response.choices[0].message.content
    filename = validate_and_trim_filename(initial_filename) + extension
    return filename

def validate_and_trim_filename(initial_filename):
    """Ensures the filename contains only allowed characters and is within length limits."""
    if not initial_filename:
        timestamp = time.strftime('%Y%m%d%H%M%S', time.gmtime())
        return f'empty_file_{timestamp}'

    cleaned_filename = re.sub(r'[^A-Za-z0-9_]', '', initial_filename)
    return cleaned_filename[:50]  # Ensure max length is 50 chars

def extract_text_from_file(filepath):
    """Extracts text from supported file types."""
    extension = os.path.splitext(filepath)[1].lower()

    if extension == ".pdf":
        return extract_text_from_pdf(filepath)
    elif extension in {".txt", ".m", ".py"}:
        return extract_text_from_text_based_file(filepath)
    elif extension == ".ipynb":
        return extract_text_from_ipynb(filepath)
    else:
        return f"This is a {extension} file. OpenAI can process structured content directly."

def extract_text_from_pdf(filepath):
    """Extracts text from a PDF file."""
    try:
        with open(filepath, 'rb') as file:
            reader = PdfReader(file)
            content = reader.pages[0].extract_text() if reader.pages else "Empty file"
        return content
    except Exception as e:
        print(f"⚠️ Error reading PDF: {e}")
        return None

def extract_text_from_text_based_file(filepath):
    """Extracts text from TXT, MATLAB, and Python files."""
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except Exception as e:
        print(f"⚠️ Error reading {filepath}: {e}")
        return None

def extract_text_from_ipynb(filepath):
    """Extracts code and markdown from a Jupyter Notebook (.ipynb) file."""
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            notebook_data = json.load(file)
        return "\n".join(cell["source"] for cell in notebook_data["cells"] if "source" in cell)
    except Exception as e:
        print(f"⚠️ Error reading Jupyter Notebook: {e}")
        return None

def select_folder():
    """Opens a folder selection dialog and renames all supported files in the selected folder."""
    folder = filedialog.askdirectory(title="Select Directory")
    if not folder:
        messagebox.showerror("Error", "No directory selected.")
        return

    print(f"📂 Selected directory: {folder}")
    
    files = [f for f in os.listdir(folder) if os.path.splitext(f)[1].lower() in supported_formats]
    if not files:
        messagebox.showinfo("No Files", "No supported files found in the selected directory.")
        return

    for file in files:
        rename_file(os.path.join(folder, file))

    messagebox.showinfo("Success", f"Renamed files in {folder}!")

# Create Tkinter GUI
root = TkinterDnD.Tk()
root.title("File Renamer")
root.geometry("500x450")

# ChatGPT Model Options
available_models = ["gpt-3.5-turbo", "gpt-3.5-turbo-0125", "gpt-4o-mini", "gpt-4","o1-preview"]
selected_model = tk.StringVar(root)  # Must be initialized after Tkinter root
selected_model.set(available_models[0])  # Default model

# Model Selection Dropdown
model_label = tk.Label(root, text="Choose ChatGPT Model:")
model_label.pack(pady=5)
model_dropdown = ttk.Combobox(root, textvariable=selected_model, values=available_models, state="readonly")
model_dropdown.pack(pady=5)

drop_label = tk.Label(root, text="Drag and Drop Files Here", bg="lightgray", width=50, height=5)
drop_label.pack(pady=10)
drop_label.drop_target_register(DND_FILES)
drop_label.dnd_bind('<<Drop>>', lambda event: rename_file(event.data.strip().strip("{}")))

select_button = tk.Button(root, text="Select Folder to Rename Files", command=select_folder)
select_button.pack(pady=10)

output_text = scrolledtext.ScrolledText(root, wrap=tk.WORD, height=10, width=60)
output_text.pack(pady=10, fill=tk.BOTH, expand=True)

sys.stdout = PrintRedirector(output_text)

root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\x.chen\Anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\x.chen\AppData\Local\Temp\ipykernel_8128\1294501579.py", line 185, in <lambda>
    drop_label.dnd_bind('<<Drop>>', lambda event: rename_file(event.data.strip().strip("{}")))
  File "C:\Users\x.chen\AppData\Local\Temp\ipykernel_8128\1294501579.py", line 61, in rename_file
    new_file_name = get_new_filename_from_openai(file_content, extension)
  File "C:\Users\x.chen\AppData\Local\Temp\ipykernel_8128\1294501579.py", line 83, in get_new_filename_from_openai
    response = client.chat.completions.create(
  File "C:\Users\x.chen\Anaconda3\lib\site-packages\openai\_utils\_utils.py", line 279, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\x.chen\Anaconda3\lib\site-packages\openai\resources\chat\completions.py", line 863, in create
    return self._post(
  File "C:\Users\x.chen\Anaconda3